In [ ]:
import threading
import time
import os
import helics as h
import pandas as pd
import config  # Import the configuration

# Set the working directory using the configuration
os.chdir(config.BASE_DIR)

# Import federates from the package
from federates import opendss_federate, voltage_consumer_federate, inverter_federate

Data Loading

In [ ]:
# Import solar production data. Remove the '_pv' suffix if present and 
# replace all occurrences of capital "S" with lower-case "s".
solar_data = pd.read_csv(f"{config.DATA_DIR}/solar_data.csv")
solar_data.columns = solar_data.columns.str.replace('_pv$', '', regex=True)
solar_data.columns = solar_data.columns.str.replace('S', 's')
solar_data['time'] = solar_data.index

# Compute the maximum solar production for each node.
# Assume node names are the columns in solar_data except for 'time'
node_names = [col for col in solar_data.columns if col != 'time']
max_solar = solar_data[node_names].max()

# Create a single-row DataFrame where column names are the node names
# and the single row contains the max solar production (used as SBAR per node).
max_solar_df = pd.DataFrame([max_solar])

# Save this DataFrame to a CSV file in the data folder.
output_csv_path = os.path.join(config.DATA_DIR, "max_solar_production.csv")
max_solar_df.to_csv(output_csv_path, index=False)
print("Max solar production per node saved to", output_csv_path)

# Immediately read back the file to create the sbar_df DataFrame.
sbar_df = pd.read_csv(output_csv_path)

# Import load data. Convert any capital "S" in the column names to lower-case.
load_data = pd.read_csv(f"{config.DATA_DIR}/load_data.csv")
load_data.columns = load_data.columns.str.replace('S', 's')
load_data['time'] = load_data.index
load_data.sort_values('time', inplace=True)

# Import the solar voltage breakpoints data and convert all capital "S" to lower-case.
breaking_points = pd.read_csv(f"{config.DATA_DIR}/solar_VV_breakpoints.csv")
breaking_points.columns = breaking_points.columns.str.replace('_pv$', '', regex=True)
breaking_points.columns = breaking_points.columns.str.replace('S', 's')

HELICS Broker Setup

In [ ]:
def start_broker():
    global broker
    broker = h.helicsCreateBroker("zmq", "", "--federates=3 --loglevel=warning")

broker_thread = threading.Thread(target=start_broker, daemon=True)
broker_thread.start()
time.sleep(1)  # Allow broker to initialize

Running the Federates

In [ ]:
# Launch the voltage consumer federate in its own thread.
consumer_thread = threading.Thread(
    target=voltage_consumer_federate.run_voltage_consumer_federate,
    args=(solar_data, load_data, node_names, config.SIMULATION_TIME, config.TIME_STEP)
)

# Launch the OpenDSS federate in its own thread.
opendss_thread = threading.Thread(target=opendss_federate.run_opendss_federate)

# Launch the inverter federate in its own thread.
# Pass both the breakpoints DataFrame and the sbar_df (node-specific SBAR values).
inverter_thread = threading.Thread(
    target=inverter_federate.run_inverter_federate,
    args=(node_names, config.SIMULATION_TIME, config.TIME_STEP, breaking_points, sbar_df)
)

# Start federates.
consumer_thread.start()
time.sleep(1.0)  # Ensure the consumer starts publishing before OpenDSS starts.
opendss_thread.start()
time.sleep(0.5)  # Optional delay for proper initialization.
inverter_thread.start()

# Wait for all federate threads to complete.
consumer_thread.join()
opendss_thread.join()
inverter_thread.join()

# =============================================================================
# Shutdown Broker
# =============================================================================
if 'broker' in globals() and h.helicsBrokerIsConnected(broker):
    h.helicsBrokerDisconnect(broker)
    h.helicsBrokerFree(broker)

print("Simulation complete. Broker closed.")


Plotting

In [ ]:
# Load the voltage timeseries data
df = pd.read_csv("voltage_timeseries.csv")

# --- Choose Your Plotting Option ---

# None for all nodes, array of strings for specific nodes
nodes_to_plot = ["701a", "701b", "701c", "727a", "727b", "727c"]
#nodes_to_plot = None

# --- Determine Which Nodes to Plot ---
if nodes_to_plot is None or len(nodes_to_plot) == 0:
    # Automatically choose all columns except the "time" column.
    nodes_to_plot = [col for col in df.columns if col != "time"]
else:
    # Check that each specified node exists in the DataFrame.
    missing = [n for n in nodes_to_plot if n not in df.columns]
    if missing:
        print(f"[ERROR] The following nodes are missing in the CSV: {missing}")
        print("Available nodes:", list(df.columns))
        exit()

# --- Plotting ---
plt.figure(figsize=(10, 6))
for node in nodes_to_plot:
    plt.plot(df['time'], df[node], label=node)

plt.xlabel("Time [s]")
plt.ylabel("Voltage Magnitude [pu]")
plt.title("Voltage Magnitude Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()